In [1]:
import numpy as np
import pandas as pd

from xbbg import blp, const, pipeline

# Reference Data `BDP` and `BDS`

In [2]:
blp.__version__

'0.7.5a5'

In [3]:
blp.bdp('AAPL US Equity', flds=['Security_Name', 'Last_Price'])

,security_name,last_price
AAPL US Equity,Apple Inc,132.69


In [4]:
blp.bdp('6758 JP Equity', flds='Crncy_Adj_Mkt_Cap', Eqy_Fund_Crncy='USD')

,crncy_adj_mkt_cap
6758 JP Equity,"125,678.20"


In [5]:
holders = blp.bds('AMZN US Equity', flds='All_Holders_Public_Filings', cache=True)
(
    holders
    .loc[:, ~holders.columns.str.contains(
        'holder_id|portfolio_name|change|number|'
        'metro|percent_of_portfolio|source'
    )]
    .rename(
        index=lambda tkr: tkr.replace(' Equity', ''),
        columns={
            'holder_name_': 'holder',
            'position_': 'position',
            'filing_date__': 'filing_dt',
            'percent_outstanding': 'pct_out',
            'insider_status_': 'insider',
        }
    )
).head()

,holder,position,filing_dt,insider,pct_out,institution_type_,country
AMZN US,Bezos Jeffrey P,"53,220,618.00",2020-12-02,Y,10.61,Unclassified,n/a
AMZN US,Vanguard Group Inc/The,"32,783,886.00",2020-09-30,N-P,6.53,Investment Advisor,United States
AMZN US,BlackRock Inc,"27,661,214.00",2020-09-30,N-P,5.51,Investment Advisor,United States
AMZN US,Scott MacKenzie,"19,786,240.00",2019-08-01,N-P,3.94,Unclassified,n/a
AMZN US,FMR LLC,"18,657,381.00",2020-12-31,N-P,3.72,Investment Advisor,United States


In [6]:
blp.dividend('SPY US Equity', start_date='2019')

,dec_date,ex_date,rec_date,pay_date,dvd_amt,dvd_freq,dvd_type
SPY US Equity,2020-12-17,2020-12-18,2020-12-21,2021-01-29,1.58,Quarter,Income
SPY US Equity,2020-09-17,2020-09-18,2020-09-21,2020-10-30,1.34,Quarter,Income
SPY US Equity,2020-06-18,2020-06-19,2020-06-22,2020-07-31,1.37,Quarter,Income
SPY US Equity,2020-03-19,2020-03-20,2020-03-23,2020-04-30,1.41,Quarter,Income
SPY US Equity,2019-12-19,2019-12-20,2019-12-23,2020-01-31,1.57,Quarter,Income
SPY US Equity,2019-09-19,2019-09-20,2019-09-23,2019-10-31,1.38,Quarter,Income
SPY US Equity,2019-06-20,2019-06-21,2019-06-24,2019-07-31,1.43,Quarter,Income
SPY US Equity,2019-03-14,2019-03-15,2019-03-18,2019-04-30,1.23,Quarter,Income


In [7]:
blp.earning('FB US Equity', Eqy_Fund_Year=2018, Number_Of_Periods=2)

,segment_name,level,fy2019,fy2018,fy2019_pct,fy2018_pct
FB US Equity,US & Canada,1,"32,206.00","25,727.00",45.55,46.07
FB US Equity,Europe,1,"16,826.00","13,631.00",23.80,24.41
FB US Equity,Asia-Pacific,1,"15,406.00","11,733.00",21.79,21.01
FB US Equity,Rest of World,1,"6,259.00","4,747.00",8.85,8.50


# Historical Data

Historical data with Excel compatible overrides

In [8]:
blp.bdh(
    tickers='SHCOMP Index', flds=['high', 'low', 'last_price'],
    start_date='2019-11', end_date='2020', Per='W', Fill='P', Days='A',
)

SHCOMP Index                    
                   high      low last_price
2019-11-01     2,980.13 2,917.15   2,958.20
2019-11-08     3,008.31 2,962.84   2,964.18
2019-11-15     2,949.96 2,891.20   2,891.34
2019-11-22     2,933.99 2,873.99   2,885.29
2019-11-29     2,915.04 2,858.58   2,871.98
2019-12-06     2,912.01 2,857.32   2,912.01
2019-12-13     2,969.98 2,902.79   2,967.68
2019-12-20     3,039.38 2,958.71   3,004.94
2019-12-27     3,036.11 2,960.43   3,005.03

Dividend / split adjustments

In [9]:
pd.concat([
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=True, CshAdjAbnormal=True, CapChg=True
    ).rename(columns={'Px_Last': 'Px_Adj'}),
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=False, CshAdjAbnormal=False, CapChg=False
    ).rename(columns={'Px_Last': 'Px_Raw'}),
], axis=1)

AAPL US Equity       
                   Px_Adj Px_Raw
2014-06-05          20.87 647.35
2014-06-06          20.81 645.57
2014-06-09          21.14  93.70
2014-06-10          21.27  94.25

# Intraday Bars

In [10]:
cur_dt = pd.Timestamp('today', tz='America/New_York').date()
recent = pd.bdate_range(end=cur_dt, periods=2, tz='America/New_York')
pre_dt = max(filter(lambda dd: dd < cur_dt, recent))
pre_dt.date()

datetime.date(2020, 12, 31)

In [11]:
blp.bdib('QQQ US Equity', dt=pre_dt, session='day').tail()

QQQ US Equity                                                      
                                   open   high    low  close   volume num_trds          value
2020-12-31 15:55:00-05:00        313.62 313.71 313.52 313.69   248779     1317  78,021,848.00
2020-12-31 15:56:00-05:00        313.69 313.89 313.69 313.88   166509      940  52,248,948.00
2020-12-31 15:57:00-05:00        313.89 314.02 313.83 313.91   302438     1647  94,943,712.00
2020-12-31 15:58:00-05:00        313.88 313.97 313.82 313.91   296271     1494  92,996,616.00
2020-12-31 15:59:00-05:00        313.92 314.24 313.57 313.74  1059753     3740 332,712,192.00

In [12]:
blp.bdib('388 HK Equity', dt=pre_dt, session='am_open_7')

388 HK Equity                                                    
                                   open   high    low  close  volume num_trds         value
2020-12-31 09:30:00+08:00        425.80 426.40 425.40 426.00  102300      140 43,568,700.00
2020-12-31 09:31:00+08:00        426.40 427.80 426.20 427.20    9400       36  4,012,060.00
2020-12-31 09:32:00+08:00        427.80 427.80 426.80 427.20   97600      153 41,678,700.00
2020-12-31 09:33:00+08:00        426.80 429.00 426.80 428.20   90600      141 38,781,060.00
2020-12-31 09:34:00+08:00        428.80 430.60 428.60 430.60  161700      225 69,512,560.00
2020-12-31 09:35:00+08:00        430.60 431.40 430.00 431.00   40800       78 17,577,380.00
2020-12-31 09:36:00+08:00        431.40 432.00 431.00 431.40   47400       77 20,454,480.00
2020-12-31 09:37:00+08:00        431.00 431.00 429.40 429.60   27500       56 11,834,160.00

# Intraday Tick Data

In [13]:
blp.bdtick('QQQ US Equity', dt=pre_dt).tail(10)

QQQ US Equity                             
                                    typ  value volume      cond exch
2020-12-31 15:00:00-05:00         TRADE 312.88    100     R6,IS    Q
2020-12-31 15:00:00-05:00         TRADE 312.88    149     R6,IS    P
2020-12-31 15:00:00-05:00         TRADE 312.88      1  R6,IS,OL    X
2020-12-31 15:00:00-05:00         TRADE 312.89     41        OL    Z
2020-12-31 15:00:00-05:00         TRADE 312.88     17        OL    B
2020-12-31 15:00:00-05:00         TRADE 312.89    500       NaN    P
2020-12-31 15:00:00-05:00         TRADE 312.89    200     R6,IS    Q
2020-12-31 15:00:00-05:00         TRADE 312.89    200     R6,IS    Q
2020-12-31 15:00:00-05:00         TRADE 312.89    200     R6,IS    K
2020-12-31 15:00:00-05:00         TRADE 312.89    200     R6,IS    Z

# Equity Screen `BEQS`

In [14]:
blp.beqs('Core Capital Ratios', typ='GLOBAL').iloc[:5, :6]

,ticker,short_name,market_cap,tier_1_cap_rt_lf,tot_rsk_bsd_cap_rt_lf,moody's_issuer_rtg
8331 JP,8331 JP,CHIBA BANK LTD,"4,488,526,848.00",11.89,12.44,A1
8355 JP,8355 JP,SHIZUOKA BANK,"4,359,666,688.00",16.17,16.17,A1
ABN NA,ABN NA,ABN AMRO BANK-CV,"9,208,644,608.00",19.00,23.40,A1
ACA FP,ACA FP,CREDIT AGRICOLE,"36,767,424,512.00",14.10,18.20,Aa3
BAC US,BAC US,BANK OF AMERICA,"262,205,423,616.00",13.50,16.10,A2


# Subscription

`blp.live` will yield market data as `dict`

In [15]:
async for snap in blp.live(['ESA Index', 'NQA Index'], info=const.LIVE_INFO, max_cnt=2):
    print(snap)

{'TICKER': 'NQA Index', 'FIELD': 'LAST_PRICE', 'MKTDATA_EVENT_TYPE': 'SUMMARY', 'MKTDATA_EVENT_SUBTYPE': 'INITPAINT', 'BID': 12882.25, 'ASK': 12887.0, 'LAST_PRICE': 12885.5, 'VOLUME': 277076, 'MID': 12884.63, 'SPREAD_BA': 4.75, 'IS_DELAYED_STREAM': True, 'REALTIME_PERCENT_BID_ASK_SPREAD': 0.03690000000000282, 'RT_PX_CHG_PCT_1D': 0.3425999879837036}
{'TICKER': 'ESA Index', 'FIELD': 'LAST_PRICE', 'MKTDATA_EVENT_TYPE': 'SUMMARY', 'MKTDATA_EVENT_SUBTYPE': 'INITPAINT', 'BID': 3748.5, 'ASK': 3749.0, 'LAST_PRICE': 3748.75, 'VOLUME': 917212, 'MID': 3748.75, 'SPREAD_BA': 0.5, 'IS_DELAYED_STREAM': True, 'REALTIME_PERCENT_BID_ASK_SPREAD': 0.013300000000000978, 'RT_PX_CHG_PCT_1D': 0.6578999757766724}


# Pipelines

In [16]:
fx = blp.bdib('JPY Curncy', dt=pre_dt)
jp = pd.concat([
    blp.bdib(ticker, dt=pre_dt, session='day')
    for ticker in ['7974 JP Equity', '9984 JP Equity']
], axis=1)
jp.tail()

7974 JP Equity                                         ... 9984 JP Equity                  \
                                    open      high       low     close   volume  ...            low    close volume   
2020-12-30 14:54:00+09:00      65,980.00 66,000.00 65,940.00 65,970.00 2,600.00  ...       8,075.00 8,079.00  32000   
2020-12-30 14:55:00+09:00      65,940.00 65,950.00 65,920.00 65,950.00 3,200.00  ...       8,078.00 8,080.00  45400   
2020-12-30 14:56:00+09:00      65,940.00 65,940.00 65,880.00 65,880.00 4,400.00  ...       8,076.00 8,078.00  46200   
2020-12-30 14:57:00+09:00      65,900.00 65,920.00 65,880.00 65,910.00 2,600.00  ...       8,077.00 8,083.00  49600   
2020-12-30 14:58:00+09:00      65,910.00 65,930.00 65,900.00 65,920.00 4,300.00  ...       8,078.00 8,081.00  59400   

                                                   
                          num_trds          value  
2020-12-30 14:54:00+09:00       60 258,512,704.00  
2020-12-30 14:55:00+09:00       78 366,882,688.00  
2020-12-30 14:56:00+09:00       91 373,219,200.00  
2020-12-30 14:57:00+09:00      111 400,801,600.00  
2020-12-30 14:58:00+09:00      110 479,994,720.00  

[5 rows x 14 columns]

Get `close` prices and convert to USD

In [17]:
prices = (
    jp
    .pipe(pipeline.get_series, col='close')
    .pipe(pipeline.apply_fx, fx=fx)
    .tz_convert('Asia/Tokyo')
)
prices.tail()

,7974 JP Equity,9984 JP Equity
2020-12-30 14:54:00+09:00,638.56,78.20
2020-12-30 14:55:00+09:00,638.37,78.21
2020-12-30 14:56:00+09:00,637.69,78.19
2020-12-30 14:57:00+09:00,637.92,78.23
2020-12-30 14:58:00+09:00,637.96,78.21


## Customized Pipelines

VWAP for intraday bar data

In [18]:
def vwap(data: pd.DataFrame, fx=None, name=None) -> pd.Series:
    return pd.Series({
        ticker: (
            data[ticker][['close', 'volume']].prod(axis=1).sum()
            if fx is None else (
                data[ticker].close
                .pipe(pipeline.apply_fx, fx)
                .close
                .mul(data[ticker].volume)
                .sum()
            )
        ) / data[ticker].volume.sum()
        for ticker in data.columns.get_level_values(0).unique()
    }, name=name)

VWAP in local currency

In [19]:
jp.pipe(vwap, name=jp.index[-1].date())

7974 JP Equity   66,033.79
9984 JP Equity    8,032.60
Name: 2020-12-30, dtype: float64

VWAP in USD

In [20]:
jp.pipe(vwap, fx=fx, name=jp.index[-1].date())

7974 JP Equity   638.34
9984 JP Equity    77.68
Name: 2020-12-30, dtype: float64

Total traded volume as of time in day for past few days

In [21]:
jp_hist = pd.concat([
    pd.concat([
        blp.bdib(ticker, dt=dt, session='day')
        for ticker in ['7974 JP Equity', '9984 JP Equity']
    ], axis=1)
    for dt in pd.bdate_range(end='today', periods=10)[:-1]
], sort=False)
unique(jp_hist.index.date)

array([datetime.date(2020, 12, 21), datetime.date(2020, 12, 22),
       datetime.date(2020, 12, 23), datetime.date(2020, 12, 24),
       datetime.date(2020, 12, 25), datetime.date(2020, 12, 28),
       datetime.date(2020, 12, 29), datetime.date(2020, 12, 30)],
      dtype=object)

In [22]:
def drop_zeros(data: pd.DataFrame) -> pd.DataFrame:
    return (
        data
        .replace(0, np.nan)
        .dropna(how='all')
        .replace(np.nan, 0)
    )

In [23]:
def traded_volume(data: pd.DataFrame, asof: str) -> pd.DataFrame:
    return (
        data
        .pipe(pipeline.get_series, col='volume')
        .between_time('0:00', asof)
        .resample('B')
        .sum()
        .pipe(drop_zeros)
    )

In [24]:
jp_hist.pipe(traded_volume, asof='10:00')

,7974 JP Equity,9984 JP Equity
2020-12-21 00:00:00+09:00,"443,500.00",3868900
2020-12-22 00:00:00+09:00,"480,500.00",3702500
2020-12-23 00:00:00+09:00,"331,600.00",3981400
2020-12-24 00:00:00+09:00,"219,200.00",6847500
2020-12-25 00:00:00+09:00,"110,200.00",6366800
2020-12-28 00:00:00+09:00,"250,100.00",5274400
2020-12-29 00:00:00+09:00,"328,200.00",3325300
2020-12-30 00:00:00+09:00,"347,200.00",3665700


In [25]:
jp_hist.pipe(traded_volume, asof='11:00')

,7974 JP Equity,9984 JP Equity
2020-12-21 00:00:00+09:00,"627,200.00",6054000
2020-12-22 00:00:00+09:00,"681,000.00",5594700
2020-12-23 00:00:00+09:00,"531,700.00",6849000
2020-12-24 00:00:00+09:00,"297,700.00",9501200
2020-12-25 00:00:00+09:00,"226,000.00",8100000
2020-12-28 00:00:00+09:00,"350,200.00",7074800
2020-12-29 00:00:00+09:00,"459,500.00",5590500
2020-12-30 00:00:00+09:00,"432,200.00",5847900
